# Popular Figures Identification in Historic Movies via Neural-Network

In [1]:
#Import of Libraries

#Image Processing
import scipy.sparse.linalg
from PIL import Image as Image, ImageDraw as Imk
from IPython.display import Image, display
import scipy 

#Dilib Libraries and skImage
import dlib as dlib
from sklearn.datasets import load_svmlight_file
from skimage.io import imread_collection
import skimage as sk
import skimage.io as io
import os
from os import listdir,makedirs
from os.path import isfile,join
import os.path
import pickle

#Dealing with Files
import cv2
import glob
from google_images_download import google_images_download 
import csv

#Face Recognintion
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

#Math 
import math
from sklearn import neighbors
import tensorflow as tf
import numpy as np
import time
from collections import Counter
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy.core.defchararray as np_f
from collections import Counter
from sklearn.metrics import confusion_matrix

#TensorFlow
from tensorflow.examples.tutorials.mnist import input_data

epsilon = 1e-10

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


The purpose of this notebook is to built a model for recognizing public figures Jean Paul Belmonod, Bridgitte Bardot, Elisabeth Reine Mere and Charles De Gaulle in historic films. A neural network is trained on a set of images and tested on videos.

## Extract Training Data From Images

The Training Data was generated from Google Image data. The images were selected by size and quality. Large and frontal faces are suited best. For the purpose of increasing the accuracy, images were choosen in which the depicted people are about the same age as in the videos. In the following code, the face_recognition library encoding was applied to individual images. One encoding per celebrity is used. 

In [2]:
#Extract encoding from each image
#This is a test, once it works, we can apply it to frames extracted from video 
img_dir = "Train_ImagefromVideo" #Enter Directory of all images 
data_path = os.path.join(img_dir,'*jpg')
files = glob.glob(data_path)
loaded_image = []
encodings = []
labels = []
i = 0
for filepath in files:
    img = face_recognition.load_image_file(filepath)
    ims = img[:, :, ::-1]
    loaded_image = np.asarray(ims)
    enco = face_recognition.face_encodings(loaded_image)
    ## Checking if there is only one encoding, if there is more/less, then the image will not be included in the testing
    if len(enco) == 1:
        ## todo: either keep this indexing method or keep separate running tally (so as not to have missing vals)
        labels.append(filepath.split("/")[1].split("_")[0])
        encodings.append(enco[0])

encodings = np.asarray(encodings)
labels = np.asarray(labels)

After loading the encodings, the arrays are stacked and brought into the right shape. Each image encoding consists our of 128 features. Therefore the shape of the data is (n, p), where n is the number of images and p is the number of features. In order to facilitate the anlysys of the data and the extraction of a CSV File, the data array generated is converted to a CSV file.  

In [3]:
# Convert the numpy array depicted above into a pandas df to facility export as csv
Celebrity_Face_Encoding = pd.DataFrame(encodings)

# Create labels dataframe
labelsDF = pd.DataFrame(labels)
labels_onehot = pd.get_dummies(labelsDF)
y_train = labels_onehot.values

In [4]:
# Take a look at the first ten values to confirm results
Celebrity_Face_Encoding.head(10)

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.024127,0.085475,0.017361,0.022435,-0.131138,0.042254,-0.014221,-0.107343,0.041117,-0.110221,...,0.068879,0.020501,-0.026679,-0.007188,-0.131504,-0.166882,-0.096925,-0.127127,0.066999,0.025766
1,-0.057144,0.026751,0.126350,-0.026666,-0.109886,-0.011346,0.022626,-0.121540,0.188620,-0.085064,...,-0.003703,0.063036,-0.019939,-0.112282,-0.055040,-0.141482,0.047746,-0.010327,0.019579,0.072041
2,-0.047625,0.075023,0.056803,-0.001579,-0.146133,0.013198,-0.054715,-0.195797,0.097233,-0.120349,...,0.059238,0.067262,-0.068762,-0.009893,-0.059077,-0.137586,-0.055399,-0.111467,0.100648,0.043378
3,-0.112403,0.122914,0.057335,0.035374,-0.072395,-0.039396,-0.029620,-0.147109,0.160966,-0.017160,...,0.061651,0.058171,0.041382,0.129432,-0.113444,-0.163540,-0.026752,-0.059040,-0.061267,0.051140
4,-0.013125,0.004420,0.018207,0.023401,-0.067221,0.007791,-0.016641,-0.077355,0.109896,-0.100729,...,0.057111,0.079056,-0.005817,0.026445,-0.178581,-0.150615,-0.056889,-0.044019,0.036203,0.043336
5,-0.082264,-0.001531,0.035755,-0.008192,-0.027778,0.003319,-0.031883,-0.149291,0.085397,-0.116676,...,0.043613,0.115006,-0.000633,0.030152,-0.184271,-0.192568,-0.047035,-0.016045,0.013198,0.111671
6,-0.033157,-0.008132,0.018028,0.013672,-0.050992,-0.003994,-0.032090,-0.089819,0.075177,-0.082718,...,0.049820,0.062390,-0.013631,0.049777,-0.188123,-0.177836,-0.057881,-0.019701,0.009197,0.040356
7,-0.163669,0.058064,0.046579,0.036525,-0.126451,0.033183,0.003657,-0.179700,0.114085,-0.111724,...,0.064204,0.169654,-0.013303,0.005778,-0.130710,-0.114165,0.052537,-0.080669,0.045099,0.068817
8,-0.111632,0.027450,0.082345,-0.026996,-0.118008,0.039777,-0.004962,-0.171604,0.208198,-0.097224,...,0.018261,0.136349,-0.000828,0.018019,-0.109773,-0.101723,0.013157,-0.061758,0.043738,0.046051
9,-0.149899,0.116029,0.037701,-0.041780,-0.089034,0.014660,0.014557,-0.145252,0.151758,-0.134705,...,0.060797,0.026797,0.029355,0.093102,-0.190019,-0.154799,0.035351,-0.072074,0.023827,0.100968


### Export CSV

In [6]:
# Export celebrity face encodings as a CSV File -> comment out so it does not create a new file each time its run
'''
Celebrity_Face_Encoding.to_csv("CSV_Files/X_train.csv")
labels_onehot.to_csv("CSV_Files/y_train.csv")
'''

The final training data set is created here.

In [8]:
# For the purposes of easier reading

X_train = encodings
# y_train already defined

### Converting images to correct coloring

In [ ]:
# In case some images aren't converted to the right coloring
# I used this when debugging for the encoding -> it may not be necessary
# If not necessary do not run as it creates overhead

# Note: Turns out not to be necessary

path = r'Video7_Frames' # Source Folder
dstpath = r'RGBConverted' # Destination Folder

try:
    #check if the specified already exists
    makedirs(dstpath)
except:
    #if it does exist write images to the same fodler
    print ("Directory already exists, images will be written in same folder")

# Folder won't use
files = [f for f in listdir(path) if isfile(join(path,f))] 

for image in files:
    try:
        img = cv2.imread(os.path.join(path,image))
        #convert to right color
        rgb_frame = img[:, :, ::-1]
        #place image to path
        dstPath = join(dstpath,image)
        #write change colored frames to the new specified directory
        cv2.imwrite(dstPath,rgb_frame)
    except:
        #exception in case the conversion was not succesful
        print ("{} is not converted".format(image))

Directory already exists, images will be written in same folder
frame11920.jpg is not converted


### Create test set

In [8]:
#Extract encoding from each image

img_dir = "Test_Images" #Enter Directory of all images 
data_path = os.path.join(img_dir,'*jpg')
files = glob.glob(data_path)
loaded_image = []
encodings = []
labels = []
i = 0
for filepath in files:
    img = face_recognition.load_image_file(filepath)
    ims = img[:, :, ::-1]
    loaded_image = np.asarray(ims)
    enco = face_recognition.face_encodings(loaded_image)
    '''
    for i in enco:
        print("enco length:", len(enco))
        print(i)
    '''
    ## Checking if there is only one encoding, if there is more/less, then the image will not be included in the testing
    if len(enco) == 1:
        labels.append(filepath)
        encodings.append(enco)

encodings = np.asarray(encodings)
labels = np.asarray(labels)


In [9]:
t = np_f.replace(labels, 'Test_Images/', '')
filepath_labels_fixed=[i.split('_')[0] for i in t] 

In [10]:
encodings.resize(encodings.shape[0], 128)
X_test = encodings
X_test_DF = pd.DataFrame(encodings)
y_test_DF = pd.DataFrame(filepath_labels_fixed)
y_test_DF_onehot = pd.get_dummies(y_test_DF)
y_test = y_test_DF_onehot.values


We will manually select images and label them in order to create our test data set. This is done as the labeling is even challenging for us as humans. Furthermore, we will remove any images that do not contain the right number of faces.  

### Export CSV

In [11]:
#Export celebrity face encodings as a CSV File -> comment out so it does not create a new file each time its run
'''
X_test_DF.to_csv("CSV_Files/X_test.csv")
y_test_DF_onehot.to_csv("CSV_Files/y_test.csv")
'''

### Skeleton for training neural net

In [69]:
'''X_train = pd.DataFrame.from_csv("CSV_Files/Celebrity_Face_Encoding_nolabel.csv")
y_train = pd.DataFrame.from_csv("CSV_Files/Labels_face_encoding.csv")

X_test = pd.DataFrame.from_csv("CSV_Files/Celebrity_Face_Encoding_nolabel_test.csv")
y_test = pd.DataFrame.from_csv("CSV_Files/Labels_face_encoding_test.csv")'''

'X_train = pd.DataFrame.from_csv("CSV_Files/Celebrity_Face_Encoding_nolabel.csv")\ny_train = pd.DataFrame.from_csv("CSV_Files/Labels_face_encoding.csv")\n\nX_test = pd.DataFrame.from_csv("CSV_Files/Celebrity_Face_Encoding_nolabel_test.csv")\ny_test = pd.DataFrame.from_csv("CSV_Files/Labels_face_encoding_test.csv")'

In [27]:
model_path = "models/faceNN.ckpt"
def facialNN(X_train, y_train, X_test, y_test, learning_rate = 0.1, training_epochs = 100, batch_size = 128, display_step = 1):
    # Parameters

    # tf Graph Input:  face encoding of length 128
    x = tf.placeholder(tf.float32, [None, 128], name='InputData')
    # how many people we're looking to classify:
    # 4 historical figures
    y = tf.placeholder(tf.float32, [None, 4], name='LabelData')

    # Set model weights
    # Note: no hidden layers here
    W = tf.Variable(tf.zeros([128, 4]), name='Weights')
    b = tf.Variable(tf.zeros([4]), name='Bias')

    # Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
    with tf.name_scope('Model'):
        # Model
        pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
    with tf.name_scope('Loss'):
        # Minimize error using cross entropy
        # We use tf.clip_by_value to avoid having too low numbers in the log function
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
    with tf.name_scope('Adam'):
        # Gradient Descent
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    with tf.name_scope('Accuracy'):
        # Accuracy
        acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        acc = tf.reduce_mean(tf.cast(acc, tf.float32))

    # Initializing the variables
    init = tf.global_variables_initializer()
    # Create a summary to monitor cost tensor
    tf.summary.scalar("Loss", cost)
    # Create a summary to monitor accuracy tensor
    tf.summary.scalar("Accuracy", acc)
    # Save results at a checkpoint (model_path)
    saver = tf.train.Saver()
    
    #STEP 2 

    # Launch the graph for training
    with tf.Session() as sess:
        sess.run(init)
        # Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.
            
            # Run optimization op (backprop), cost op (to get loss value) and summary nodes
            _, c = sess.run([optimizer, cost], feed_dict={x: X_train, y: y_train})
            
            # Display logs per epoch step
            if (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(c))

        print("Optimization Finished!")

        # Test model
        # Calculate accuracy
        accRate = acc.eval({x: X_test, y: y_test})
        
        print("Accuracy rate:", accRate)
        save_path = saver.save(sess, model_path)
        
        '''# To test unknown faces
        prediction=tf.argmax(y,1)
        predList = prediction.eval(feed_dict={y: y_test})
        '''
        #print (predList)
        return model_path

In [30]:
facialNN(X_train, y_train, X_test, y_test)

Epoch:  01   =====> Loss= 1.386294365
Epoch:  02   =====> Loss= 1.249869347
Epoch:  03   =====> Loss= 0.975380540
Epoch:  04   =====> Loss= 0.766343415
Epoch:  05   =====> Loss= 0.675655723
Epoch:  06   =====> Loss= 0.561120033
Epoch:  07   =====> Loss= 0.450502574
Epoch:  08   =====> Loss= 0.380107999
Epoch:  09   =====> Loss= 0.330492914
Epoch:  10   =====> Loss= 0.283273935
Epoch:  11   =====> Loss= 0.238732189
Epoch:  12   =====> Loss= 0.203358799
Epoch:  13   =====> Loss= 0.178692415
Epoch:  14   =====> Loss= 0.159978658
Epoch:  15   =====> Loss= 0.142220587
Epoch:  16   =====> Loss= 0.124285191
Epoch:  17   =====> Loss= 0.107715257
Epoch:  18   =====> Loss= 0.094033822
Epoch:  19   =====> Loss= 0.083542123
Epoch:  20   =====> Loss= 0.075547650
Epoch:  21   =====> Loss= 0.069027178
Epoch:  22   =====> Loss= 0.063149333
Epoch:  23   =====> Loss= 0.057505317
Epoch:  24   =====> Loss= 0.052076556
Epoch:  25   =====> Loss= 0.047051929
Epoch:  26   =====> Loss= 0.042632364
Epoch:  27  

'models/faceNN.ckpt'

In [18]:
def readImageFiles(img_dir):
    #Extract encoding from each image
    #This is a test, once it works, we can apply it to frames extracted from video 
    data_path = os.path.join(img_dir,'*.jpg')
    files = glob.glob(data_path)
    loaded_image = []
    encodings = []
    labels = []
    i = 0
    for filepath in files:
        img = face_recognition.load_image_file(filepath)
        ims = img[:, :, ::-1]
        loaded_image = np.asarray(ims)
        enco = face_recognition.face_encodings(loaded_image)

        ## Checking if there is only one encoding, if there is more/less, then the image will not be included in the testing
        if len(enco) == 1:
            ## todo: either keep this indexing method or keep separate running tally (so as not to have missing vals)
            labels.append(filepath)
            encodings.append(enco)

    encodings = np.asarray(encodings)
    labels = np.asarray(labels)
    return encodings, labels

In [19]:
def readVideo(filename, model):
    # Open the input movie file
    input_movie = cv2.VideoCapture(filename)
    #input_movie = cv2.VideoCapture("MGAFE0410353.01.mp4")
    length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
    y = int(input_movie.get(3))
    #print("Y",y)
    x = int(input_movie.get(4))
    #print("X",x)

    # Create an output movie file (make sure resolution/frame rate matches input video!)
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    cleanFilename = filename[:-4]
    #cleanFilename = filename + "_output.avi"
    cleanFilename = model + "_" + filename.split('/')[1] + "_output.avi"
    output_movie = cv2.VideoWriter(cleanFilename, fourcc = fourcc, fps=24.0, frameSize=(y, x))
    # Load some sample pictures and learn how to recognize them.
    #Deep Learning Library face_encoding called
    
    return length, input_movie, output_movie

In [20]:
def nnToVideo(filename):
    
    length, input_movie, output_movie = readVideo(filename, "nn")
    
    known_faces, labels = readImageFiles("Train_ImagefromVideo")
    
    # Initialize some variables
    face_locations = []
    face_encodings = []
    face_names = []
    frame_number = 0
    saver = tf.train.Saver()
    x = tf.placeholder(tf.float32, [None, 128], name='InputData')
    y = tf.placeholder(tf.float32, [None, 4], name='LabelData')

    
    #while input_movie.isOpened():
    while True: 
        # Grab a single frame of video
        ret, frame = input_movie.read()
        #print(frame.shape)
        frame_number += 1

        # Quit when the input video file ends
        if not ret:
            break

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_frame = frame[:, :, ::-1]

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        face_names = []
        for face_encoding in face_encodings:
            print("Found a face")
            # See if the face is a match for the known face(s)
            
            # could make this logic a lot prettier
            name = None
            with tf.Session() as sess:
                saver.restore(sess, model_path)
                prediction=tf.argmax(y,1)
                predList = prediction.eval(feed_dict={y: face_encoding})
                #print("Accuracy:", acc.eval({x: test_xs, m: test_ms, y: test_ys}))
            #predList = facialNN(X_train, y_train, X_test, y_test)
                                
            if predList[0] == 0:
                name = "BrigitteBardot"
            elif predList[0] == 1:
                name = "CharlesDeGaulle"
            elif predList[0] == 2:
                name = "ElizabethReineMere"
            elif predList[0] == 3:
                name = "JeanPaulBelmondo"

            face_names.append(name)

        # Label the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            if not name:
                continue

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

        # Write the resulting image to the output video file
        print("Writing frame {} / {}".format(frame_number, length))

        # confirmed: frames are being read fine, the issue is writing each frame
        output_movie.write(frame)

    print("All done")
    # All done!
    input_movie.release()
    output_movie.release()
    cv2.destroyAllWindows()
    

In [21]:
nnToVideo("Test_Videos/MGAFE0008218--AS.01.mp4")

ValueError: No variables to save